In [6]:
from transformers import AutoTokenizer
from zero_shot_re import RelTaggerModel, RelationExtractor

model = RelTaggerModel.from_pretrained("fractalego/fewrel-zero-shot")
tokenizer = AutoTokenizer.from_pretrained("fractalego/fewrel-zero-shot")

#### Premier essai

In [7]:
relations = ['container', 'content', 'other relation']
extractor = RelationExtractor(model, tokenizer, relations)
cubes='the cubes'
box='the box'
ranked_rels = extractor.rank(text='the box must be filled with the cubes', head=box, tail=cubes)
print(ranked_rels)

[('container', 0.9636894315481186), ('content', 0.719708263874054), ('other relation', 0.3384249806404114)]


#### Tester la relation de contenance entre deux items

In [14]:
relations = ['container/content', 'other relation']
extractor = RelationExtractor(model, tokenizer, relations)
def contenance(text,item1,item2):
    #print(text,",",item1,",",item2)
    ranked_rels = extractor.rank(text=text, head=item1, tail=item2)
    #print(ranked_rels)
    if ranked_rels[1][1]>0.5:
        return 'other relation'
    elif ranked_rels[0][1]>0.9:
        return 'container/content'
    else:
        return 'other relation'

In [17]:
text='You must put the cubes and the spheres in the boxes'
box='the boxes'
cube='the cubes'
sphere='the spheres'
print(contenance(text,box,cube))
print(contenance(text,cube,sphere))
print(contenance(text,box,sphere))

container/content
other relation
container/content


In [18]:
text='the cubes must be filled with boxes and spheres'
box='boxes'
cube='the cubes'
sphere='spheres'
print(contenance(text,box,cube))
print(contenance(text,cube,sphere))
print(contenance(text,box,sphere))

container/content
container/content
other relation


#### Trouver l'ensemble de toutes les relations de contenance

In [19]:
from itertools import combinations
import spacy

nlp = spacy.load("en_core_web_lg")

def get_chunk(sent):
    doc=nlp(sent)
    nps=doc.noun_chunks
    sentences=[]
    for np1, np2 in combinations([np for np in nps], 2):
        sentences.append((sent,np1.text,np2.text))
    return sentences

def get_rel(sent):
    relations={}
    for (sent,item1,item2) in get_chunk(sent):
        relations[(item1,item2)]=contenance(sent,item1,item2)
    return relations

In [20]:
get_rel('the boxes must be filled with cubes and spheres')

{('the boxes', 'cubes'): 'container/content',
 ('the boxes', 'spheres'): 'container/content',
 ('cubes', 'spheres'): 'other relation'}

In [23]:
get_rel("the treasure is in the chest of the car")

{('the treasure', 'the chest'): 'container/content',
 ('the treasure', 'the car'): 'other relation',
 ('the chest', 'the car'): 'other relation'}